In [151]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

In [152]:
import pandas as pd
        
acoes_listadas_b3 = pd.read_csv('acoes-listadas-b3.csv', sep = ',')

In [153]:

# Configuração inicial do WebDriver
service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920x1080")
options.add_argument("--disable-notifications")
options.add_argument('--no-sandbox')
options.add_argument('--verbose')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

In [154]:

# Inicie o navegador
browser = webdriver.Chrome(service=service, options=options)

# Lista de URLs das ações
acoes_urls = ['https://investidor10.com.br/acoes/taee11/', 'https://investidor10.com.br/acoes/wege3/', 'https://investidor10.com.br/acoes/petr4/']

df = pd.DataFrame()

In [155]:
acoes_listadas_b3['Ticker']

In [156]:

for urls in acoes_listadas_b3['Ticker']:
    url = 'https://investidor10.com.br/acoes/' + str(urls).lower() + '/'
    try:
        browser.get(url)
        time.sleep(10)  # Wait for the page to load completely
        page_content = browser.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        table = soup.find(id='table-indicators-history')

        if table:
            years = [th.get_text().strip() for th in table.find_all("th", class_="year")]
            indicadores = [td.get_text().strip() for td in table.find_all("td", class_="indicator")]

            all_values = []
            for row in table.find_all("tr", class_=["visible-even", "visible-odd"]):
                values = [value_td.get_text().strip() for value_td in row.find_all("td", class_="value")]
                if len(values) == len(years):
                    all_values.append(values)

            df_temp = pd.DataFrame(all_values, columns=years, index=indicadores)
            df_temp['acao'] = str(urls).lower()
            df = pd.concat([df, df_temp])

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")

        break  # Break the loop after handling the error

df.to_csv("investidor10_acoes.csv")
browser.quit()

In [160]:
df.shape

In [161]:
df.to_csv("investidor10_acoes_1.csv")